In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.stats import kendalltau 
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.datasets import fetch_california_housing
import torch
import matplotlib.cm as cm
from itertools import combinations
from joblib import Parallel, delayed
import os

# data loading
data = fetch_california_housing(as_frame=True)
df = data.frame


def generate_data(n, m):
    sampled_data = df.sample(n=n, replace=True)
    v, w = sampled_data["MedInc"].to_numpy(), sampled_data["MedHouseVal"].to_numpy()
    x = sampled_data.drop(columns=["MedInc", "MedHouseVal"]).values

    sampled_data_unlabel = df.sample(n=m, replace=True)
    x_unlabel = sampled_data_unlabel.drop(columns=["MedInc", "MedHouseVal"]).values

    return x, x_unlabel, v, w


def Ucross(x, x_unlabel, v, w):
    
    x_a, x_b, v_a, v_b, w_a, w_b = train_test_split(x, v, w, test_size=0.5)
    x_unlabel_a, x_unlabel_b = train_test_split(x_unlabel, test_size=0.5)

    def fit_model(x, y):
        model = XGBRegressor()
        model.fit(x, y)
        return model
    
    def compute_ecdfs(v_split, w_split):
        n = len(v_split)
        ecdf_v = np.sum(v_split[:, None] <= v_split, axis=0) / n
        ecdf_w = np.sum(w_split[:, None] <= w_split, axis=0) / n
        ecdf_vw = np.sum((v_split[:, None] <= v_split) & (w_split[:, None] <= w_split), axis=0) / n
        return ecdf_v, ecdf_w, ecdf_vw

    def compute_ell1(ecdf_v, ecdf_w, ecdf_vw):
        return (1 - 2 * ecdf_v) * (1 - 2 * ecdf_w) + 4 * (ecdf_vw - ecdf_v * ecdf_w)

    ecdf_v_a, ecdf_w_a, ecdf_vw_a = compute_ecdfs(v_a, w_a)
    ecdf_v_b, ecdf_w_b, ecdf_vw_b = compute_ecdfs(v_b, w_b)

    ell1_a = compute_ell1(ecdf_v_a, ecdf_w_a, ecdf_vw_a)
    ell1_b = compute_ell1(ecdf_v_b, ecdf_w_b, ecdf_vw_b)
    ell1_cross = np.concatenate((ell1_a, ell1_b))

    model_a = fit_model(x_a, ell1_a)
    fhat_label_b = model_a.predict(x_b)
    fhat_unlabel_b = model_a.predict(x_unlabel_b)
    
    model_b = fit_model(x_b, ell1_b)
    fhat_label_a = model_b.predict(x_a)
    fhat_unlabel_a = model_b.predict(x_unlabel_a)
    
    fhat_label_cross = np.concatenate((fhat_label_a, fhat_label_b))
    fhat_label_unlabel_cross = np.concatenate((fhat_label_cross, fhat_unlabel_a, fhat_unlabel_b))
    
    # Ucross stat
    U = kendalltau(v, w)[0]
    Ucross = U - np.mean(fhat_label_cross) + np.mean(fhat_label_unlabel_cross)
    
    gamma1 = np.mean((ell1_a - np.mean(ell1_a)) * (fhat_label_a - np.mean(fhat_label_a))) \
             / np.var(np.concatenate((fhat_label_a, fhat_unlabel_a)))
    gamma2 = np.mean((ell1_b - np.mean(ell1_b)) * (fhat_label_b - np.mean(fhat_label_b))) \
             / np.var(np.concatenate((fhat_label_b, fhat_unlabel_b)))
             
    Ucross_cv = U - np.mean(np.concatenate((gamma1 * fhat_label_a, gamma2 * fhat_label_b))) \
                         + np.mean(np.concatenate((gamma1 * fhat_label_a, gamma2 * fhat_label_b, gamma1 * fhat_unlabel_a, gamma2 * fhat_unlabel_b)))
    
    
    # variance 
    n = len(x)
    m = len(x_unlabel)
    
    Ui = []
    for i in range(n):
        v_leave_one_out = np.delete(v, i)
        w_leave_one_out = np.delete(w, i)
        Ui.append(kendalltau(v_leave_one_out, w_leave_one_out)[0])

    Ui = np.array(Ui)
    sigma2 = (n - 1) / 4 * np.sum((Ui - U)**2)
    tau = np.mean((fhat_label_cross - ell1_cross - np.mean(fhat_label_cross - ell1_cross))**2) - sigma2
    var_U = 4 * sigma2
    var_Ucross = 4 * sigma2 + 4 * m / (n + m) * tau
    return U, Ucross, var_U, var_Ucross   
# population kendall tau
pop_kendall_tau = kendalltau(df["MedInc"], df["MedHouseVal"])[0]


def simulation(n, m):
    x, x_unlabel, v, w = generate_data(n, m)
    U, Uss, var_U, var_Uss = Ucross(x, x_unlabel, v, w)

    lb = U - 1 / np.sqrt(n) * norm.ppf(0.975) * np.sqrt(var_U)
    ub = U + 1 / np.sqrt(n) * norm.ppf(0.975) * np.sqrt(var_U)

    lb_ss = Uss - 1 / np.sqrt(n) * norm.ppf(0.975) * np.sqrt(var_Uss)
    ub_ss = Uss + 1 / np.sqrt(n) * norm.ppf(0.975) * np.sqrt(var_Uss)

    count_in_ci = ((lb <= pop_kendall_tau) & (pop_kendall_tau <= ub)).sum()
    count_in_ci_ss = ((lb_ss <= pop_kendall_tau) & (pop_kendall_tau <= ub_ss)).sum()

    avg_length_ci = (ub - lb).mean()
    avg_length_ci_ss = (ub_ss - lb_ss).mean()

    return count_in_ci, count_in_ci_ss, avg_length_ci, avg_length_ci_ss


results_dict = {}
n_seq = [1000, 1200, 1400, 1600, 1800, 2000]

for n in n_seq:
    m = n * 10
    results = Parallel(n_jobs=-1)(delayed(simulation)(n, m) for _ in tqdm(range(10000), desc=f"n={n}"))
    count_in_ci_list, count_in_ci_ss_list, avg_length_ci_list, avg_length_ci_ss_list = zip(*results)

    results_dict[n] = {
        "count_in_ci": np.mean(count_in_ci_list),
        "count_in_ci_ss": np.mean(count_in_ci_ss_list),
        "avg_length_ci": np.mean(avg_length_ci_list),
        "avg_length_ci_ss": np.mean(avg_length_ci_ss_list),
        "se_length_ci": np.std(avg_length_ci_list, ddof=1) / np.sqrt(len(avg_length_ci_list)),
        "se_length_ci_ss": np.std(avg_length_ci_ss_list, ddof=1) / np.sqrt(len(avg_length_ci_ss_list)),
    }



# Use LaTeX for text rendering and set professional font
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["cmr10"],
    "axes.titlesize": 20,
    "axes.labelsize": 20,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 16
})

color_u = "#0072BD"  
color_ucross = "#D95319" 

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Average Count Plot (Coverage Probability)
axes[0].plot(n_seq, [results_dict[n]["count_in_ci"] for n in n_seq], 
             marker='o', markersize=7, linestyle='-', linewidth=2, color=color_u, label=r'$U$')
axes[0].plot(n_seq, [results_dict[n]["count_in_ci_ss"] for n in n_seq], 
             marker='s', markersize=7, linestyle='-', linewidth=2, color=color_ucross, label=r'$U_{\mathrm{cross}}$')
axes[0].axhline(y=0.95, color='gray', linestyle='--', linewidth=1.2)
axes[0].set_xlabel(r'$n$')
axes[0].set_ylabel(r'Coverage')
axes[0].set_ylim(0.8, 1.0)
axes[0].legend(loc='lower left')
axes[0].set_title(r'Coverage Probability of Confidence Interval')
axes[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.7)  # Add gridlines

# Average Length Plot (without error bars)
axes[1].plot(n_seq, [results_dict[n]["avg_length_ci"] for n in n_seq],
             marker='o', markersize=7, linestyle='-', linewidth=2, color=color_u, label=r'$U$')
axes[1].plot(n_seq, [results_dict[n]["avg_length_ci_ss"] for n in n_seq],
             marker='s', markersize=7, linestyle='-', linewidth=2, color=color_ucross, label=r'$U_{\mathrm{cross}}$')
axes[1].set_xlabel(r'$n$')
axes[1].set_ylabel(r'Width')
axes[1].legend(loc='lower left')
axes[1].set_title(r'Average Width of Confidence Interval')
axes[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.7)  # Add gridlines

plt.tight_layout()

# Ensure the directory exists
output_dir = os.path.join(os.path.expanduser('~'), 'Desktop')
os.makedirs(output_dir, exist_ok=True)
plt.savefig(os.path.join(output_dir, 'CI_Housing.pdf'), format='pdf')
plt.show()

print(f"PDF: {os.path.join(output_dir, 'CI_Housing.pdf')}")